In [1]:
def connect_to_db():

    import sqlalchemy
    import sqlite3
    
    # Create SQL Alchemy database connection
    db_url = f"sqlite:///C:/Users/JanseDanny/OneDrive/Documenten Danny/Football Fortunes/database/footballfortunes.sqlite"
    ##db_url = f"sqlite:////app/data/database/DEV/footballfortunes.sqlite"
    engine = sqlalchemy.create_engine(db_url, echo=True)
    sqal_connection = engine.connect()

    # Create Psychop connection
    psy_connection = sqlite3.connect("C:/Users/JanseDanny/OneDrive/Documenten Danny/Football Fortunes/database/footballfortunes.sqlite")
    ##psy_connection = sqlite3.connect("/app/data/database/DEV/footballfortunes.sqlite")
    
    return psy_connection, sqal_connection

def close_connection_db(psy_connection, sqal_connection):
    
    # Commit de transactie en sluit de verbinding
    sqal_connection.close()
    psy_connection.close()   
    

In [26]:
def betgenerator(sqal_connection):

    from datetime import datetime
    import pandas as pd
    
    print('betgenerator gestart')
    # psy_cursor = psy_connection.cursor()
    
    huidige_datum = datetime.now().strftime("%d%m%Y_%I%M%S")
    
    # Lees het SQL-bestand in en voer de queries uit
    sql_file = '../07 Betgenerator/sql/betgenerator.sql'
    
    with open(sql_file) as file:
        sql_script = file.read()
        betadvies = pd.read_sql(sql_script, sqal_connection)
    
    betadvies.to_excel(f'C:\\Users\\JanseDanny\\OneDrive\\Documenten Danny\\Football Fortunes\\Bet_advies\\test\\betadvies_{huidige_datum}.xlsx', index=False, engine='openpyxl')


    

In [27]:
sqal_connection = connect_to_db()[1]

betgenerator(sqal_connection)

betgenerator gestart
2024-10-23 15:58:54,017 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-23 15:58:54,018 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("WITH odds AS (
SELECT hometeam, awayteam, odd_datum, max(odd_prijs) best_odd
FROM dwa_f_odds
GROUP BY 1,2,3
HAVING odd_datum > current_date
),
actuele_vorm AS (
SELECT * FROM dwa_f_vorm
WHERE nr = 1
),
advies_berekening AS ( 
select
dt.seizoen
,o.odd_datum
,ddw.competitie
,ddw.hometeam 
,ddw.awayteam 
,o.best_odd
,vhome.GR_na GR_home
,vaway.GR_na GR_away
,(vhome.GR_na - vaway.GR_na) GRO
,(vhome.SR_na - vaway.SR_na) SRO
,ddb.a
,ddb.rc_GRO 
,ddb.rc_GRO
,ddb.min_GRO
,ddb.max_GRO
,ddb.min_SRO
,ddb.max_SRO
,ddb.dif
,1/(ddb.a + ddb.rc_GRO * (vhome.GR_na - vaway.GR_na)  + ddb.rc_SRO * (vhome.SR_na - vaway.SR_na)) + dif min_odd
FROM odds o
LEFT JOIN DWA_d_wedstrijden ddw 
	ON o.hometeam = ddw.hometeam AND o.awayteam = ddw.awayteam 
LEFT JOIN DWA_d_betparameters ddb 
	ON ddw.competitie = ddb.competitie
LEFT JOIN dwa_d_datum dt 